# Exercices 3 - AMDA Catalogs with Speasy

<div align="center">
<img src="https://raw.githubusercontent.com/SciQLop/speasy/main/logo/logo_speasy.svg"/>
    <img src="http://cdpp.irap.omp.eu/images/logosOutils/amda.png" />
</div>

In [ ]:
import speasy as spz

# Downloading AMDA catalogs

Speasy catalog objects are basically a list of time intervals plus some metadata.

Get the LPP bowshock list from AMDA shared catalogs located in **Catalogs/SharedCatalogs/EARTH/LPP_bowshock_list** using `speasy.get_data`.

In [ ]:
#
#
# your code here
#
# bs_catalog = .....
#

In [ ]:
# answer
amda = spz.inventories.data_tree.amda
bow_shock_catalog_path = amda.Catalogs.SharedCatalogs.EARTH.LPP_bowshock_list
bs_catalog = spz.get_data(bow_shock_catalog_path)

--- 

# Getting catalog informations

Now you have a catalog called `bs_catalog`, can you tell who shared this catalog online, who is the contact name, what mission events are from, and what is the overall time interval this catalog is made over

In [ ]:
#
#
# your code here
#
#

In [ ]:
# answer : 
for k, v in bs_catalog.meta.items():
    print(f"{k}: {v}")

---

# Manipulating events - plotting them

Below is a function that plots the omnidirectional energy flux, the ion bulk velocity and the magnetic field from MMS1, the same quantities as in exercice 2. : 

- mms1_dis_energyspectr_omni_brst
- mms1_dis_bulkv_gse_brst
- mms1_fgm_b_gse_brst_l2


In case there is no burst data for the event (hint: the len() of the time attribute is 0), then take the 

- mms1_dis_energyspectr_omni_fast
- mms1_dis_bulkv_gse_fast
- mms1_fgm_b_gse_srvy_l2

The following function takes a `dataset` that is a list of 3 `SpeasyVariable` for these quantities and saves figures on disk.
The goal of this exercice is to call this function so to make the same figure from exercice 2 for the events in the AMDA catalog just downloaded for which the Y coordinate of the crossing is such that $|Y| < 0.2 R_e$


In [ ]:
import matplotlib.pyplot as plt
%matplotlib widget
from matplotlib.colors import LogNorm

def plot_event(dataset):
    """
    Parameter:
    """
    fig, axes = plt.subplots(nrows=3, figsize=(10,5), sharex=True)
    spectro  = datasets[0]
    bulkVel  = datasets[1]
    magnetic = datasets[2]
    
    axes[0].pcolormesh(spectro.time,
                       spectro.axes[1].values[0,:],
                       spectro.values.T, cmap="jet", norm=LogNorm())
    axes[0].set_yscale("log")
    
    axes[1].plot(bulkVel.time, bulkVel.values, label=bulkVel.columns)
    axes[2].plot(magnetic.time, magnetic.values, label = magnetic.columns)
    
    for ax in axes[1:]:
        ax.legend(ncol=4, loc="best")
        
    fig.tight_layout()
    fig.savefig(f"{spectro.time[0]}_{spectro.time[-1]}.png")
    plt.close(fig)

In [ ]:
#
#
# your code here
#
#

In [ ]:
# ANSWER
# ------
import numpy as np
cda = spz.inventories.data_tree.cda
for event in bs_catalog:
    
    to_select = np.abs(event.meta["Col 2"])< 0.2
    
    if to_select:
        
        mms1_prods = [cda.MMS.MMS1.DIS.MMS1_FPI_BRST_L2_DIS_MOMS.mms1_dis_energyspectr_omni_brst,
                            cda.MMS.MMS1.DIS.MMS1_FPI_BRST_L2_DIS_MOMS.mms1_dis_bulkv_gse_brst,
                            cda.MMS.MMS1.FGM.MMS1_FGM_BRST_L2.mms1_fgm_b_gse_brst_l2]

        datasets = spz.get_data(mms1_prods, event.start_time, event.stop_time)
        
        if len(datasets[0].time)==0:
            mms1_prods = [cda.MMS.MMS1.DIS.MMS1_FPI_FAST_L2_DIS_MOMS.mms1_dis_energyspectr_omni_fast,
                          cda.MMS.MMS1.DIS.MMS1_FPI_FAST_L2_DIS_MOMS.mms1_dis_bulkv_gse_fast,
                          cda.MMS.MMS1.FGM.MMS1_FGM_SRVY_L2.mms1_fgm_b_gse_srvy_l2]
        
            datasets = spz.get_data(mms1_prods, event.start_time, event.stop_time)        

        plot_event(datasets)